In [54]:
import pandas as pd
import numpy as np
from StringIO import StringIO
import csv
import requests
import requests_cache
import tarfile
import urllib
from datetime import datetime
from datetime import timedelta

In [7]:
# SWPC's events for a given day are stored in a separate file, and all the files
# for the year 2013 are in a compressed archive. Each of the files looks like 
# the one below.

test_str = """
:Product: 20121229events.txt
:Created: 2013 Jan 01 0332 UT
:Date: 2012 12 29
# Prepared by the U.S. Dept. of Commerce, NOAA, Space Weather Prediction Center
# Please send comments and suggestions to SWPC.Webmaster@noaa.gov 
#
# Missing data: ////
# Updated every 30 minutes.
#                            Edited Events for 2012 Dec 29
#
#Event    Begin    Max       End  Obs  Q  Type  Loc/Frq   Particulars       Reg#
#-------------------------------------------------------------------------------

 180       0557   0557      0557  LEA  G   RBR  410       200                   

 190       1203   1209      1214  G15  5   XRA  1-8A      B6.3    3.0E-04   1638

 200       1927   1933      1939  G15  5   XRA  1-8A      C1.5    7.3E-04   1638
 200       1929   1931      1940  HOL  3   FLA  N12E51    SF      ERU       1638
"""

# Note that the separators in the above file are spaces, not tabs. We will
# therefore need to develop functionality that converts files like these into
# a format that can be recognized by pandas.

def split_and_purge(in_str, delim=None, purge=["", "+"]):
    """Splits a string into an array of strings, removing unwanted strings."""
    pieces = []
    if delim is None:
        pieces = in_str.split()  # Split for both spaces and tabs.
    else:
        pieces = in_str.split(delim)
    return [piece for piece in pieces if piece not in purge]

def parse_events(events_file):
    """Parses an swpc events file into a pandas-friendly form."""
    # Split into lines.
    lines = split_and_purge(events_file, delim="\n")
    
    # Throw away the first 10 lines. We do not need the info contained therein.
    lines = lines[10:]

    if len(lines) < 2:
        return ""

    # Get the column titles. Clean them.
    title_row = split_and_purge(lines[0])
    title_row = [title.replace("#", "") for title in title_row]
    
    # We merge the Particulars and Reg columns since these are sometimes left
    # blank. The other fields do not seem to have blank values (based on
    # spot-checking).
    title_row = title_row[:-1]
    
    # Get the rows.
    rows = []
    for line in lines[2:]:
        items = split_and_purge(line)
        
        # Too few items.
        if len(items) < 8:
            continue
        
        # Take the first 8 columns as distinct items. Remaining items are
        # thrown together into a single column.
        rows.append(items[:8] + [" ".join(items[8:])])

    # Create csv.
    strio = StringIO()
    writer = csv.writer(strio)
    writer.writerows([title_row] + rows)
    return strio.getvalue()

# Test the function above.
pd.read_csv(StringIO(parse_events(test_str)))

,Event,Begin,Max,End,Obs,Q,Type,Loc/Frq,Particulars
0,180,557,557,557,LEA,G,RBR,410,200
1,190,1203,1209,1214,G15,5,XRA,1-8A,B6.3 3.0E-04 1638
2,200,1927,1933,1939,G15,5,XRA,1-8A,C1.5 7.3E-04 1638
3,200,1929,1931,1940,HOL,3,FLA,N12E51,SF ERU 1638


In [51]:
# We proceed to read in all SWPC events for the year 2012 into a dataframe.
import re
#SWPC_2012_URL = "http://legacy-www.swpc.noaa.gov/ftpdir/warehouse/2012/2012_events.tar.gz"
SWPC_2012_URL = '/Users/nschanch/Downloads/2012_events.tar.gz'
#strio = StringIO(requests.get(SWPC_2012_URL).content)
#strio = StringIO(SWPC_2012_URL)
#tarred = tarfile.AREGTYPEopen("r:gz", fileobj=strio)
tarred = tarfile.open(SWPC_2012_URL)
print strio
swpc_by_day = []
for tarinfo in tarred:
    if not tarinfo.isdir():
        extracted = tarred.extractfile(tarinfo).read()
        day = re.match(r"^2012_events/(\d+)events.txt", tarinfo.name).group(1)
        df = pd.read_csv(StringIO(parse_events(extracted)))
        df["Date"] = [day] * len(df)
        swpc_by_day.append(df)

# Form a single dataframe. Replace '////'
swpc = pd.concat(swpc_by_day)
for col in list(swpc.columns):
    swpc[col] = swpc[col].apply(lambda x: np.nan if "////" in str(x) else x)

swpc.sort(["Date", "Begin"], ascending=[1, 1], inplace=True)
swpc.reset_index(drop=True, inplace=True)
swpc.head()

<StringIO.StringIO instance at 0x1078b97e8>


,Event,Begin,Max,End,Obs,Q,Type,Loc/Frq,Particulars,Date
0,3710,0,NaN,1052,LEA,C,RSP,031-180,CTM/1,20111229
1,3600,316,0319,326,G15,5,XRA,1-8A,C1.2 5.6E-04,20111229
2,3600,323,NaN,737,LEA,C,RSP,025-180,VI/2,20111229
3,3610,358,0405,411,G15,5,XRA,1-8A,C3.0 1.4E-03 1389,20111229
4,3610,404,0405,409,LEA,3,FLA,S26E72,SF 1389,20111229


In [52]:
# Next, we read SWPC Sunspot Data, similarly to above.

sunspot_test = """
:Product: 0105SRS.txt
:Issued: 2013 Jan 05 0030 UTC
# Prepared jointly by the U.S. Dept. of Commerce, NOAA,
# Space Weather Prediction Center and the U.S. Air Force.
#
Joint USAF/NOAA Solar Region Summary
SRS Number 5 Issued at 0030Z on 05 Jan 2013
Report compiled from data received at SWO on 04 Jan
I.  Regions with Sunspots.  Locations Valid at 04/2400Z 
Nmbr Location  Lo  Area  Z   LL   NN Mag Type
1638 N13W34   308  0080 Hsx  02   01 Alpha
1640 N28W48   322  0320 Eki  13   24 Beta-Gamma-Delta
1641 N04W05   279  0040 Cao  08   02 Beta
1642 S24E22   252  0100 Hsx  02   01 Alpha
1644 N15E35   239  0040 Hsx  02   01 Alpha
1645 S13W18   292  0050 Dao  05   08 Beta
1646 N14E45   229  0030 Hax  01   01 Alpha
1647 N16W55   329  0030 Cao  05   04 Beta
1648 N05E53   221  0020 Cro  03   02 Beta
1649 S14E74   200  0060 Hsx  02   01 Alpha
1650 S26E77   197  0050 Hsx  02   01 Alpha
1651 N19E48   225  0010 Axx  01   01 Alpha
IA. H-alpha Plages without Spots.  Locations Valid at 04/2400Z Jan
Nmbr  Location  Lo
1636  N14W69   343
1637  N06W77   351
1639  S16W38   312
1643  S14E17   257
II. Regions Due to Return 05 Jan to 07 Jan
Nmbr Lat    Lo
None
"""

# Numpy returns NotImplemented when you try to compare lists of
# strings for equality. This is just insane. We provide here an
# alternative for all(np.equal(a, b))
def are_equal(list_a, list_b):
    if len(list_a) != len(list_b):
        return False
    return all([a == b for a, b in zip(list_a, list_b)])


sunspot_title_row = ['Nmbr', 'Location', 'Lo', 'Area', 'Z', 'LL', 'NN', 'Mag', 'Type']

def parse_sunspot(sunspot_file):
    """Parses a swpc sunspot file into a pandas-friendly form."""
    # Split into lines.
    lines = split_and_purge(sunspot_file, "\n")

    # Look for the first row that has 9 words. This is the title row.
    i = -1
    title_row = None
    for line in lines:
        i += 1
        if are_equal(sunspot_title_row, split_and_purge(line)):
            title_row = sunspot_title_row
            title_row = title_row[:7] + [" ".join(title_row[7:])]
            break
        else:
            continue

    if title_row is None:
        return ""

    # Get the rows.
    rows = []
    for line in lines[i + 1:]:
        items = split_and_purge(line)

        # We are done retrieving the rows we need.
        if len(items) != 8:
            break

        rows.append(items)

    # Create csv.
    strio = StringIO()
    writer = csv.writer(strio)
    writer.writerows([title_row] + rows)
    return strio.getvalue()

# Test the function above.
pd.read_csv(StringIO(parse_sunspot(sunspot_test))) 


,Nmbr,Location,Lo,Area,Z,LL,NN,Mag Type
0,1638,N13W34,308,80,Hsx,2,1,Alpha
1,1640,N28W48,322,320,Eki,13,24,Beta-Gamma-Delta
2,1641,N04W05,279,40,Cao,8,2,Beta
3,1642,S24E22,252,100,Hsx,2,1,Alpha
4,1644,N15E35,239,40,Hsx,2,1,Alpha
5,1645,S13W18,292,50,Dao,5,8,Beta
6,1646,N14E45,229,30,Hax,1,1,Alpha
7,1647,N16W55,329,30,Cao,5,4,Beta
8,1648,N05E53,221,20,Cro,3,2,Beta
9,1649,S14E74,200,60,Hsx,2,1,Alpha


In [57]:
def get_sunspots(year) :
    """Returns a dataframe containing sunspot data for a given year."""
    today = np.datetime64(datetime.now())

    # Parse every file for the given year into a dataframe, then add the df
    # to a list.
    sunspot_by_day = []
    
    # The files are placed into an archive after the year ends. Before that,
    # they are uncompressed.
    if today.item().year == year:
        # Since it takes about a minute to scrape all the text files individually for
        # a given year, we save the parsed content locally as a tarfile, and load it up
        # when we want the data again.
        
        local_tar_path = os.path.join(os.getcwd(), "%d_SRS.tar.gz" % year)
        if os.path.exists(local_tar_path):  # Archive exists from recent save.
            tarred = tarfile.open(local_tar_path, "r:gz")
            for tarinfo in tarred:
                if tarinfo.isdir():
                    continue
                extracted = tarred.extractfile(tarinfo).read()

                # The tar members are named like '20100420SRS.txt'. We use
                # a regex to extract the day.
                day = re.match(r"^(\d+).txt", tarinfo.name).group(1)

                df = pd.read_csv(StringIO(extracted))
                df["Date"] = [day] * len(df)
                sunspot_by_day.append(df)
            tarred.close()
        else:  # Archive does not exists, so download files and add them to the archive.
            tarred = tarfile.open(local_tar_path, "w:gz")
            urlformat = "http://legacy-www.swpc.noaa.gov/ftpdir/warehouse/%d/SRS/%sSRS.txt"
            urlformat = '/Users/nschanch/Downloads/%s_SRS/%sSRS.txt'
            curr_date = np.datetime64(str(year))
            while curr_date < today:
                date_str = curr_date.item().strftime("%Y%m%d")
                curr_url = urlformat % (year, date_str)
                strio = StringIO(parse_sunspot(requests.get(curr_url).content))
                df = pd.read_csv(strio)
                df["Date"] = [date_str] * len(df)
                sunspot_by_day.append(df)
                info = tarred.tarinfo()
                info.name = "%s.txt" % date_str
                info.size = strio.len
                strio.seek(0)  # Reset file position.
                tarred.addfile(info, fileobj=strio)
                curr_date += np.timedelta64(1, "D")
            tarred.close()
    else:
        # The prefix for the archive file for 2006 is different from the other years.
        file_pref = "" if year == 2006 or year == 2007 else "%s_" % year

        #The swpc changed their website during the course of this project.  The data will still be
        #available at this legacy site for 60 days. After this we would need to find a new url.  
        #SUNSPOT_URL = "http://legacy-www.swpc.noaa.gov/ftpdir/warehouse/%s/%sSRS.tar.gz" % (year, file_pref)
        SUNSPOT_URL = '/Users/nschanch/Downloads/2012_SRS.tar.gz'
        #strio = StringIO(requests.get(SUNSPOT_URL).content)
        tarred = tarfile.open(SUNSPOT_URL)

        # Parse every file for the given year into a dataframe, then add the df
        # to a list.
        for tarinfo in tarred:
            if tarinfo.isdir():
                continue
            
            extracted = tarred.extractfile(tarinfo).read()

            # The tar members are named like '2010_SRS/20100420SRS.txt'. We use
            # a regex to extract the day.
            day = re.match(r"^.*?SRS/(\d+)SRS.txt", tarinfo.name).group(1)

            df = pd.read_csv(StringIO(parse_sunspot(extracted)))
            df["Date"] = [day] * len(df)
            sunspot_by_day.append(df)
        tarred.close()

    # Form a single dataframe.
    sunspot = pd.concat(sunspot_by_day)
    sunspot.sort(["Date", "Nmbr"], ascending=[1, 1], inplace=True)
    sunspot.reset_index(drop=True, inplace=True)
    return sunspot

# Get spots for year 2012.
sunspot = get_sunspots(2012)
sunspot.head()

AttributeError: 'NoneType' object has no attribute 'group'